# Match Quaia to (all) SDSS spectra.

In support of an *SDSS-V* spare-fiber proposal.

## License:
Copyright 2024 the authors all rights reserved.

## Authors:
- **Kate Storey-Fisher** (DIPC), working as volunteeer (!)
- **David W. Hogg** (Flatiron)

## Projects:
- Establish a target list for a possible *SDSS-V* spare-fiber program.
- Make plots relevant to that spare-fiber proposal.

## Bugs and known issues:
- This assumes that the DR16 "superset" file contains anything that might be relevant. This is probably false in detail.

In [ ]:
import numpy as np
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rcParams['figure.dpi'] = 150

import astropy
from astropy.table import Table, join, vstack
from astropy import units as u
from astropy.coordinates import SkyCoord

## Download and read data
This will take a while the first time you run it.

In [ ]:
fn = "quaia_G20.5.fits"
dfn = "../data/" + fn
cmd = "curl -s -o " + dfn + " https://zenodo.org/records/8060755/files/" + fn
if not os.path.isfile(dfn + ".gz"):
    print(cmd)
    os.system(cmd)
    os.system("gzip -fv --best " + dfn)
dfn = dfn + ".gz"
foo = os.system("ls -al " + dfn)

In [ ]:
quaia = Table.read(dfn)
print("Columns:", quaia.columns)
print("N =", len(quaia))

In [ ]:
fns = "DR16Q_Superset_v3.fits"
dfns = "../data/" + fns
cmd = "curl -s -o " + dfns + " https://data.sdss.org/sas/dr16/eboss/qso/DR16Q/" + fns
if not os.path.isfile(dfns + ".gz"):
    print(cmd)
    os.system(cmd)
    os.system("gzip -fv --best " + dfns)
dfns = dfns + ".gz"
foo = os.system("ls -al " + dfns)

In [ ]:
sdss = Table.read(dfns)
sdss = sdss[sdss['OBJID'].mask==False] #remove ones with no objid
print("Columns:", sdss.columns)
print("N =", len(sdss))

## Do cross-matching

In [ ]:
def cross_match(ra1, dec1, ra2, dec2, separation):
    coords1 = SkyCoord(ra=ra1, dec=dec1, frame='icrs')    
    coords2 = SkyCoord(ra=ra2, dec=dec2, frame='icrs') 
    cross = astropy.coordinates.search_around_sky(coords1, coords2, separation) 
    index_list_1in2, index_list_2in1 = cross[0], cross[1] 
    return index_list_1in2, index_list_2in1

In [ ]:
# Perform cross-match; 1 arcsec is a reasonable (but MAGIC) separation
separation = 1*u.arcsec
quaiaINsdss, sdssINquaia = cross_match(quaia['ra'], quaia['dec'],
                                    sdss['RA']*u.degree, sdss['DEC']*u.degree,
                                    separation=separation)
print(len(quaiaINsdss))

In [ ]:
# Make non-match list
quaiaNOTINsdss = np.ones(len(quaia)).astype(bool)
quaiaNOTINsdss[quaiaINsdss] = False
print(np.sum(quaiaNOTINsdss))

In [ ]:
quaia_exsdss = quaia[quaiaNOTINsdss]
print(len(quaia_exsdss))

## Make plots

In [ ]:
plt.scatter(quaia['ra'], np.sin(quaia['dec'] * np.pi / 180.), c="k", s=0.1, alpha=0.1)
plt.xlabel("RA (deg)")
plt.ylabel("sin(Dec)")
plt.title("all of Quaia")

In [ ]:
plt.scatter(quaia_exsdss['ra'], np.sin(quaia_exsdss['dec'] * np.pi / 180.), c="k", s=0.1, alpha=0.1)
plt.xlabel("RA (deg)")
plt.ylabel("sin(Dec)")
plt.title("Quaia minus sources observed previously by some SDSS")
plt.savefig("quaia_exsdss.png")